In [42]:
# import all the required packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict,OrderedDict
from sklearn import preprocessing
from sklearn.metrics.pairwise import cosine_distances, euclidean_distances
from scipy import sparse
from operator import itemgetter
from scipy.spatial.distance import cosine
import json
import pickle
import seaborn
from surprise import SVD
from surprise import KNNBasic,AlgoBase

In [43]:
# reading data from dataset
df = pd.read_csv('temp/db1.csv')
# breif about the columns in the dataset
df.head(3)
print(np.shape(df))
df.describe()

(1000, 18)


,id,phoneNo,age,minimumWage,experience,clientsAttended,aadharCard
count,1000.000000,1.000000e+03,1000.000000,1000.000000,1000.000000,1000.000000,1.000000e+03
mean,500.500000,8.771674e+09,28.375000,5901.500000,6.193000,39.988000,5.093677e+11
std,288.819436,8.437247e+08,6.520651,2465.588546,2.790975,14.971841,2.873929e+11
min,1.000000,7.001783e+09,18.000000,2000.000000,1.000000,5.000000,1.267541e+09
25%,250.750000,8.110433e+09,23.000000,3500.000000,4.000000,29.000000,2.512533e+11
50%,500.500000,9.001577e+09,28.000000,6000.000000,7.000000,43.500000,5.103412e+11
75%,750.250000,9.482632e+09,34.000000,8000.000000,9.000000,52.000000,7.495910e+11
max,1000.000000,9.997569e+09,40.000000,10000.000000,10.000000,60.000000,9.997775e+11


In [44]:
df = df.drop(['phoneNo','id','availabilityPreference','aadharCard','liscenced','shoppingliscence'],axis=1)
#df.drop_duplicates(inplace=True)
df.dropna(inplace=True)
# df.fillna(0,inplace=True)
# df.head()

In [45]:
# l = preprocessing.LabelEncoder()
# nan_rows = df[df['location'].isnull()]
# nan_rows
# creating classes for dummy purpose only
#----------------------------------------------------------------------------------------------------------------------
def classes_maker():
    for i in df.columns:
        le = preprocessing.LabelEncoder()
        le.fit(df[i])
        df[i] = le.transform(df[i])
classes_maker()
# df.head()
# le = preprocessing.LabelEncoder()
# le.fit_transform(df['location'])
# df['location']

In [46]:
# le.fit(df['certification'])
# df['certification'] = le.transform(df['certification'])
# le.fit(df['occupation'])
# df['occupation'] = le.transform(df['occupation'])
# le.fit(df['gender'])
# df['gender'] = le.transform(df['gender'])
# le.fit(df['type'])
# df['type'] = le.transform(df['type'])
# le.fit(df['availability'])
# df['availability'] = le.transform(df['availability'])
# le.fit(df['doorstepService '])
# df['doorstepService '] = le.transform(df['doorstepService '])
# le.fit(df['references'])
# df['references'] = le.transform(df['references'])
# le.fit(df['clientsAttended'])
# df['clientsAttended'] = le.transform(df['clientsAttended'])
# le.fit(df['experience'])
# df['experience'] = le.transform(df['experience'])
# df.head()
# np.shape(df)[0]
np.shape(df)
# df.head(10002)

(1000, 12)

In [47]:
occupations=defaultdict(list)
# location: index and occupation where index is the position of row in Dataframe
def all_occupations_in_a_location():
    for index,row in df.iterrows():
        occupations[row['location']].append((index,row['occupation']))
        
    for key, values in occupations.items():
        t_set = list(set(occupations.get(key)))
        occupations[key] = t_set
all_occupations_in_a_location()
# print(occupations.get(0))
# print(sorted(occupations.items()))
# print(occupations.get(0)[2])
# print(occupations[0][5])
# df.head()
# temp = occupations[0]
# temp.sort(key=itemgetter(1))
# print(temp)
# print(temp[0][1])
# indexes = []
# print(len(occupations[0]))
# for i in range(len(occupations.get(0))):
#     if i+1 < len(occupations.get(0)):
#         if temp[i][1] == temp[i+1][1]:
#             indexes.append((temp[i+1][0],temp[i][0]))
# print(indexes)
# ----------------------------------------------------------------------------------------------------
# to unpickle use 
# from ast import literal_eval
# with open(path,'r') as f:
# ...     type(pickle.loads(literal_eval(f.read())))

with open('occupations.txt','w') as f:
    f.write(str(pickle.dumps(occupations)))
print("Occupation dict has been pickled.")

Occupation dict has been pickled.


In [319]:
# cosdist = np.ndarray(shape=(np.shape(df)[0],np.shape(df)[0]))
# eucdist = np.ndarray(shape=(np.shape(df)[0],np.shape(df)[0]))
# def cos_func(index1,index2):
#     # will calcuate the cosine distance between 2 workers and return to helper
#     #print(index1,index2)
#     return cosine_distances(df.loc[index1].values.reshape(1, -1),df.loc[index2].values.reshape(1, -1))

# def euc_dist(index1,index2):
#     return euclidean_distances(df.loc[index1].values.reshape(1, -1),df.loc[index2].values.reshape(1, -1))


# def cos_helper(indexes):
#     # worker with similar occupation in a given location is passed as 'indexes'
#     #TODO: pass pair of tuples in the 'indexes' list to the cosine distance function which will return the distance between
#     # those 2 data points and will be saved in 'dist' list w.r.t index of those points so that we have distance of all workers 
#     # which we will use to make a matrix/ to cluster
#     for i in indexes:
#         for j in indexes:
#             if i != j:
#                 cosdist[int(i)][int(j)] = cos_func(i,j)
#                 eucdist[int(i)][int(j)] = euc_dist(i,j)


# # find workers at a location having a specific occupation for calculating cosine distance between them
# def commonities():
#         for k, v in occupations.items(): # location iterator
#             type(occupations[k]) # list type
#             temp_list = occupations[k]
#             indexes = np.array([])
#             temp_list.sort(key=itemgetter(1)) # sort list according to occupation
#             # print(temp_list)
                        
#             # TODO: Find similar occupations and then pass it to cosine_distance calculator
#                 #print(occupations[k][l]) #tuple type
                
#                 # this will find all workers having similar occupation in a particular location
#                 # append only the index of worker
#             for i in range(len(occupations[k])):
#                 if i+1 < len(occupations[k]):
#                     if temp_list[i][1] == temp_list[i+1][1]:
#                         indexes = np.append(indexes,temp_list[i][0])
#                         indexes = np.append(indexes,temp_list[i+1][0])
#             #print(df.loc[indexes[4]],df.loc[indexes[1]])
#             #print(len(indexes)) total number of workers with similar occupations in a given location
#             cos_helper(indexes)
                
# commonities()
# # print(dist)
# #-----------------------------------------------------------------------------------------------
# # TO UNPICKLE
# # with open(path,'r') as f:
# # ...     type(pickle.loads(literal_eval(f.read())))

# # idea is to search for the similar occupation and return its index and remove it from the list as there might be duplicates

# # return index,array
# # def cos_dist():
# #     for key in occupations.keys():
# #         for l in len():
# #             dist[index].append(cosine_distances(occupations[key].reshape(1,-1),row.reshape(1,-1))))
# print('done')

In [41]:
#Euclidean distance between items
euc_dist = euclidean_distances(sparse.csr_matrix(df))
#Cosine distance between items
cos_dist = cosine_distances(sparse.csr_matrix(df))

In [40]:
%store euc_dist
%store cos_dist

Stored 'euc_dist' (ndarray)
Stored 'cos_dist' (ndarray)


In [49]:
cos_dist

array([[ 0.        ,  0.15431467,  0.02523744, ...,  0.39762922,
         0.23352023,  0.46457129],
       [ 0.15431467,  0.        ,  0.07727058, ...,  0.40860244,
         0.41074464,  0.47503188],
       [ 0.02523744,  0.07727058,  0.        , ...,  0.36284122,
         0.26101882,  0.43543216],
       ..., 
       [ 0.39762922,  0.40860244,  0.36284122, ...,  0.        ,
         0.06580191,  0.00826849],
       [ 0.23352023,  0.41074464,  0.26101882, ...,  0.06580191,
         0.        ,  0.07870348],
       [ 0.46457129,  0.47503188,  0.43543216, ...,  0.00826849,
         0.07870348,  0.        ]])

In [50]:
euc_dist

array([[  0.        ,  28.58321186,  12.24744871, ...,  47.75981575,
         45.61797891,  49.16299421],
       [ 28.58321186,   0.        ,  18.52025918, ...,  46.84015371,
         58.54912467,  45.67274899],
       [ 12.24744871,  18.52025918,   0.        , ...,  44.50842617,
         48.77499359,  45.46427169],
       ..., 
       [ 47.75981575,  46.84015371,  44.50842617, ...,   0.        ,
         26.41968963,   8.1240384 ],
       [ 45.61797891,  58.54912467,  48.77499359, ...,  26.41968963,
          0.        ,  29.563491  ],
       [ 49.16299421,  45.67274899,  45.46427169, ...,   8.1240384 ,
         29.563491  ,   0.        ]])